In [6]:
import os
import sys
import re
from collections import defaultdict
root_dir = '/'.join(os.getcwd().split('/')[:-1])
datasets_dir = '{:}/datasets'.format(root_dir)
propbank_dir = '{:}/csvs/1.0/'.format(datasets_dir)
sys.path.insert(0, root_dir)
sys.path.insert(0, datasets_dir)
sys.path.insert(0, propbank_dir)

file_path = '{:}/gs.csv'.format(propbank_dir)

ARGUMENTS=['A0','A1','A2','A3','A4','A5',
'AM-ADV','AM-CAU','AM-DIR','AM-DIS','AM-EXT','AM-LOC',
'AM-MED','AM-MNR','AM-NEG','AM-PNC','AM-PRD','AM-REC',
'AM-TMP']
import pandas as pd 

In [7]:
df = pd.read_csv(file_path)
df.head(60) 


,INDEX,ID,S,P,P_S,FORM,LEMMA,GPOS,MORF,CTREE,PRED,ARG
0,0,1,1,1,0,Brasília,Brasília,PROP,F|S,(FCL(NP*),-,*
1,1,2,1,1,0,Pesquisa_Datafolha,Pesquisa_Datafolha,N,F|S,(NP*,-,(A0*
2,2,3,1,1,0,publicada,publicar,V-PCP,F|S,(ICL(VP*),-,*
3,3,4,1,1,0,hoje,hoje,ADV,-,(ADVP*))),-,*)
4,4,5,1,1,0,revela,revelar,V-FIN,PR|3S|IND,(VP*),revelar,(V*)
5,5,6,1,1,0,um,um,ART,M|S,(NP*,-,(A1*
6,6,7,1,1,0,dado,dado,N,M|S,*,-,*
7,7,8,1,1,0,supreendente,surpreendente,ADJ,M|S,(ADJP*),-,*
8,8,9,1,1,0,:,-,PU,-,*,-,*
9,9,10,1,1,0,recusando,recusar,V-GER,-,(FCL(ICL(VP*),-,*


In [8]:
DATASET_TRAIN_PROPOSITIONS = (1,5300)
DATASET_VALID_PROPOSITIONS = (5301,5536)
DATASET_TEST_PROPOSITIONS = (5537, 5776) 


dftrain = df[ (df['P'] >= DATASET_TRAIN_PROPOSITIONS[0]) \
              & (df['P'] <= DATASET_TRAIN_PROPOSITIONS[1]) ]

dfvalid = df[ (df['P'] >= DATASET_VALID_PROPOSITIONS[0])\
              & (df['P'] <= DATASET_VALID_PROPOSITIONS[1]) ]

dfdevelop = df[ (df['P'] >= DATASET_TRAIN_PROPOSITIONS[0]) \
              & (df['P'] <= DATASET_VALID_PROPOSITIONS[1]) ]

dftest = df[ (df['P'] >= DATASET_TEST_PROPOSITIONS[0])\
              & (df['P'] <= DATASET_TEST_PROPOSITIONS[1]) ]


In [9]:
print('====train database====')
prop_train= max(dftrain['P']) - min(dftrain['P'])
sent_train= max(dftrain['S']) - min(dftrain['S'])
verb_train=len(set(dftrain['PRED']))-1


prop_valid=max(dfvalid['P'])-min(dfvalid['P'])
sent_valid=max(dfvalid['S'])-min(dfvalid['S'])
verb_valid=len(set(dfvalid['PRED']))-1


prop_develop=max(dfvalid['P'])-min(dftrain['P'])
sent_develop=max(dfvalid['S'])-min(dftrain['S'])
verb_develop=len(set(dfvalid['PRED']).union(set(dftrain['PRED'])))-1



prop_train_perc = (float(prop_train) / prop_develop) * 100
sent_train_perc= (float(sent_train) / sent_develop)*100
verb_train_perc= (float(verb_train) / verb_develop)*100



prop_valid_perc= (float(prop_valid) / prop_develop)*100
sent_valid_perc= (float(sent_valid) / sent_develop)*100
verb_valid_perc= (float(verb_valid) / verb_develop)*100

print('\t\t\ttrain\t\tdevel\t\tvalid')
values=('Proposições',prop_develop,100.0,prop_train, prop_train_perc, prop_valid, prop_valid_perc)
buff= '%s\t%d %.2f\t%d %.2f\t%d %.2f\t' %  values
print(buff)    
values=('Sentenças',sent_develop,100.0,sent_train, sent_train_perc, sent_valid, sent_valid_perc)
buff= '%s\t%d %.2f\t%d %.2f\t%d %.2f\t' %  values
print(buff)    
values=('Verbos Distintos',verb_develop,100.0,verb_train, verb_train_perc, verb_valid, verb_valid_perc)
buff= '%s\t%d %.2f\t%d %.2f\t%d %.2f\t' %  values
print(buff)    


====train database====
			train		devel		valid
Proposições	5535 100.00	5299 95.74	235 4.25	
Sentenças	3162 100.00	3011 95.22	151 4.78	
Verbos Distintos	1000 100.00	989 98.90	148 14.80	


In [10]:
print('====test database====')
print('propositions:', max(dftest['P'])-min(dftest['P']))
print('sentences:', max(dftest['S'])-min(dftest['S']))
print('verbs distinct:', len(set(dftest['PRED']))-1)

print('====total database====')
print('propositions:', max(df['P'])-min(df['P']))
print('sentences:', max(df['S'])-min(df['S']))
print('verbs distinct:', len(set(df['PRED']))-1)


print('propositions:',(max(dftest['P'])-min(dftest['P'])) /(max(df['P'])-min(df['P']) ))



====test database====
propositions: 239
sentences: 144
verbs distinct: 165
====total database====
propositions: 5775
sentences: 3307
verbs distinct: 1023
propositions: 0.041385281385281386


In [12]:
def argument_transform(val):
        if isinstance(val, str):
            val = re.sub(r'C\-|\(|\)|\*|\\n| +', '',val)            
#             val = re.sub(r'\(|\)|\*|\\n| +', '',val)            
            val = re.sub(r' ', '',val)          
        else: 
            val=''
        return val

def argument_count(df):
    '''Computes the count for each argument
    
    '''
    argcount_dict = defaultdict(lambda : 0)
    arg_list = df.loc[:,'ARG'].values.tolist()
    for arg_ in  arg_list:
        arg_ = argument_transform(arg_)
        if arg_ in ARGUMENTS:
            argcount_dict[arg_] += 1
    return argcount_dict
    
    
    

In [13]:
stats = argument_count(df)

stats_develop= argument_count(dfdevelop)
stats_train= argument_count(dftrain)
stats_valid= argument_count(dfvalid)
stats_test=  argument_count(dftest)

devel_total=0
train_total=0
valid_total=0
# arguments=['A0','A1','A2','A3','A4','A5',
# 'AM-ADV','AM-CAU','AM-DIR','AM-DIS','AM-EXT','AM-LOC',
# 'AM-MED','AM-MNR','AM-NEG','AM-PNC','AM-PRD','AM-REC',
# 'AM-TMP']
template = '{:}\t{:04d} {:.2f}\t{:04d} {:.2f}\t{:04d} {:.2f}\t{:04d} {:.2f}'

print('ARG\t\ttotal\t\ttrain\t\tvalid\t\ttest')
for key in ARGUMENTS:
    stats_list_ = [key] 
    value = stats.get(key,0)
    perc  = 100.0
    stats_list_.append(value)
    stats_list_.append(perc)
    
    trainvalue = stats_train.get(key,0)
    trainperc  = 100*(float(trainvalue) / value) if value > 0 else 0
    stats_list_.append(trainvalue)
    stats_list_.append(trainperc)
    
    validvalue = stats_valid.get(key,0)
    validperc  = 100*(float(validvalue) / value) if value > 0 else 0
    stats_list_.append(validvalue)
    stats_list_.append(validperc)
    
    testvalue = stats_test.get(key,0)
    testperc  = 100*(float(testvalue) / value) if value > 0 else 0
    stats_list_.append(testvalue)
    stats_list_.append(testperc)
    
#     values=(key, trainvalue, trainperc, developvalue, developperc, validvalue, validperc)

    devel_total+=    trainvalue + validvalue
    train_total+=    trainvalue
    valid_total+=    validvalue
    test_total+=     testvalue
    total += devel_total + te 
    buff = template.format(*stats_list_) 
    print(buff)    
    
print('train total:',train_total,float(train_total)*100/train_total)    
print('devel total:',devel_total,float(devel_total)*100/train_total)    
print('valid total:',valid_total,float(valid_total)*100/train_total)    

print('test total:',test_total,float(test_total)*100/train_total)    


ARG		total		train		valid		test


NameError: name 'developvalue' is not defined